In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns

from funciones_varias import cargaCsvToDataFrame

dfMoviesFinal = cargaCsvToDataFrame("dfMoviesFinal", "datasets_eda")

In [ ]:
#considero que para el modelo de clasificacion no necesito estas columnas
dfMoviesFinal = dfMoviesFinal.drop(columns=["tagline", "id", "status"])

In [29]:
#dropeo estas rows porque considero que tienen generos de pelicula que no me sirven para el modelo, son datos erroneos

dfMoviesFinal.drop(dfMoviesFinal[dfMoviesFinal["movie_genre"]== "Aniplex"].index, inplace=True)
dfMoviesFinal.drop(dfMoviesFinal[dfMoviesFinal["movie_genre"]== "Odyssey Media"].index, inplace=True)
dfMoviesFinal.drop(dfMoviesFinal[dfMoviesFinal["movie_genre"]== "Carousel Productions"].index, inplace=True)

In [ ]:
df